In [89]:
import numpy as np
import scipy.linalg
import scipy.special
import matplotlib.pyplot as plt
import random
import itertools

In [3]:
def delta(a, b):
    '''
    The Kronecker delta function
    
    a = b  -> return 1
    a != b -> return 0
    '''
    if a == b:
        return 1
    return 0

In [152]:
# NOTE: The number of partcile must be 4 * n

numberOfParticles = 16
if numberOfParticles % 2 != 0:
    raise ValueError

In [153]:
# ----------------------------------
# Create the random coupling tensor
# ----------------------------------

random.seed()

N = numberOfParticles

J = np.zeros((N, N, N, N))  # Initialize the tensor with zeros
for a in range(N):          # Insert random numbers to J
    for b in range(N):
        for c in range(N):
            for d in range(N):
                J[a, b, c, d] = random.gauss(0, 6.0 / (N * N * N))

for a in range(N):          # Make the tensor J be totally anti symmetric
    for b in range(N):
        for c in range(N):
            for d in range(N):
                if a == b or a == c or a == d or b == c or b == d or c == d:
                    J[a, b, c, d] = 0
                if a < b < c < d:
                    J[a, b, d, c] = -J[a, b, c, d]
                    J[a, c, b, d] = -J[a, b, c, d]
                    J[a, d, c, b] = -J[a, b, c, d]
                    
                    J[b, a, c, d] = -J[a, b, c, d]
                    J[b, c, d, a] = -J[a, b, c, d]
                    J[b, d, a, c] = -J[a, b, c, d]
                    
                    J[c, a, d, b] = -J[a, b, c, d]
                    J[c, b, a, d] = -J[a, b, c, d]
                    J[c, d, b, a] = -J[a, b, c, d]
                    
                    J[d, a, b, c] = -J[a, b, c, d]
                    J[d, b, c, a] = -J[a, b, c, d]
                    J[d, c, a, b] = -J[a, b, c, d]
                    
                    J[a, c, d, b] = J[a, b, c, d]
                    J[a, d, b, c] = J[a, b, c, d]
                    
                    J[b, a, d, c] = J[a, b, c, d]
                    J[b, c, a, d] = J[a, b, c, d]
                    J[b, d, c, a] = J[a, b, c, d]
                    
                    J[c, a, b, d] = J[a, b, c, d]
                    J[c, b, d, a] = J[a, b, c, d]
                    J[c, d, a, b] = J[a, b, c, d]
                    
                    J[d, a, c, b] = J[a, b, c, d]
                    J[d, b, a, c] = J[a, b, c, d]
                    J[d, c, b, a] = J[a, b, c, d]

In [154]:
#
# Create an array which has state vectors in the Fock space as its entries
#
# The creation/annihilation opetators is defined by the following:
#     c_i          = (psi_2i - i psi_(2i-1)) / sqrt(2)
#     c_i^dagger   = (psi_2i + i psi_(2i-1)) / sqrt(2)
# where i runs from 1 to the half of the number of particles
#
# For example, if the number of particles is 4 then
# stateVectorArray = [[0,0], [0,1], [1,0], [1,1]]
#

N = int(numberOfParticles / 2)

stateVectorArray = []

# -----------------------------------------------------------
# How to Create the fermion state vectors
# -----------------------------------------------------------
#
# Since the number of a fermion is 0 or 1, we have states such that | 0 0 1 0 1 1 ...> and so on.
# Thus we have a  binary number 001011... for each state vector,
# and we can make a correspondece between a state representated in a binary number and a number with base 10.
# For example, let us consider a state such that only the first fermon exists: |1 0 0 0 0 ... >.
# This state corresponds to 1 in the base 10, and its binary representation is 000...00001.
# Note that we expect the way of numbering fermions as | "first", "second", "third", ... >,
# but the corresponding binary number has the reversed numbering way i.e.  000...000 "third" "second" "first".
# Thus we need to reverse the binary numbers.
#


for i in range(2 ** N):
    stateString = bin(i)
    stateString = stateString[2:]
    while len(stateString) < N:
        stateString = "0" + stateString
    stateVector = list(stateString)
    stateVector = list(map(int, stateVector))
    stateVector = stateVector[::-1]
    stateVectorArray.append(stateVector)

In [155]:
# Represent the creation/annihilation operators as matrices

def creationOperator(fermionIndex, leftStateVector, rightStateVector):
    matrixElement = 1
    for stateIndex in range(len(leftStateVector)):
        if fermionIndex == stateIndex:
            matrixElement *= delta(leftStateVector[stateIndex], rightStateVector[stateIndex] + 1)
        else:
            matrixElement *= delta(leftStateVector[stateIndex], rightStateVector[stateIndex])
    
    return matrixElement


def annihilationOperator(fermionIndex, leftStateVector, rightStateVector):
    matrixElement = 1
    for stateIndex in range(len(leftStateVector)):
        if fermionIndex == stateIndex:
            matrixElement *= delta(leftStateVector[stateIndex], rightStateVector[stateIndex] - 1)
        else:
            matrixElement *= delta(leftStateVector[stateIndex], rightStateVector[stateIndex])
    
    return matrixElement

In [156]:
# create arrays [c_0, c_1, c_2, ...] and [c_0^dagger, c_1^dagger, c_2^dagger, ...]

creationOperatorMatricesArray     = []
annihilationOperatorMatricesArray = []

for fermionIndex in range(N):
    creationOperatorMatrix = np.mat([[creationOperator(fermionIndex, l, r) for l in stateVectorArray] for r in stateVectorArray])
    annihilationOperatorMatrix = np.mat([[annihilationOperator(fermionIndex, l, r) for l in stateVectorArray] for r in stateVectorArray])
    
    # NOTE: the two indices of a matrix is transversed as default, so we need to transvers.
    creationOperatorMatricesArray.append(creationOperatorMatrix.T)
    annihilationOperatorMatricesArray.append(annihilationOperatorMatrix.T)

KeyboardInterrupt: 

In [ ]:
# representate psi's in the SYK Hamiltonian as matrices
# psi_2i = (c_i + c_i^dagger) / sqrt(2),      psi_(2i - 1) = i (c_i - c_i^dagger) / sqrt(2)

psiArray = []

for i in range(N):
    psi = (annihilationOperatorMatricesArray[i] + creationOperatorMatricesArray[i]) / np.sqrt(2)
    psiArray.append(psi)
    psi = (1J * (annihilationOperatorMatricesArray[i] - creationOperatorMatricesArray[i])) / np.sqrt(2)
    psiArray.append(psi)

In [143]:
psiArray[0]

matrix([[0.        , 0.70710678, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.70710678, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.70710678, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.70710678, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.7071

In [ ]:
# Create the SYK Hamiltonian 

# Initilize the Hamiltonian with zeros
Hamiltonian = np.mat([[0 for _ in range(2 ** N)] for _ in range(2 ** N)])

for a in range(numberOfParticles):
    for b in range(numberOfParticles):
        for c in range(numberOfParticles):
            for d in range(numberOfParticles):
                Hamiltonian = Hamiltonian + J[a, b, c, d] * psiArray[a] * psiArray[b] * psiArray[c] * psiArray[d]

Hamiltonian = Hamiltonian / scipy.special.factorial(numberOfParticles, exact=True)


In [149]:
eigenValue, eigenVector = scipy.linalg.eig(Hamiltonian)

In [150]:
eigenValue

array([-9.00523150e-07+0.00000000e+00j,  1.21251372e-06+1.84411812e-23j,
        1.22790221e-07-1.16269105e-23j, -4.34780792e-07+2.20972874e-23j,
       -9.00523150e-07+4.22259745e-23j,  1.21251372e-06-4.63575665e-23j,
       -1.44655781e-06+2.12350755e-23j,  1.22790221e-07-1.57351989e-23j,
       -4.34780792e-07+1.01750780e-23j,  3.04144168e-06+1.05879118e-22j,
        3.04144168e-06-1.50055733e-22j, -1.44655781e-06-1.08786344e-22j,
       -6.45086180e-07+9.72720943e-23j, -9.49797695e-07-9.26442275e-23j,
       -9.49797695e-07+2.50416006e-23j, -6.45086180e-07-3.97046694e-23j])

In [151]:
eigenVector

array([[ 1.00000000e+00+0.00000000e+00j, -6.27319172e-17-7.84148965e-18j,
        -1.48381822e-33-1.33108095e-16j,  2.61348737e-16+9.50359044e-17j,
        -2.91881879e-02-1.06783242e-01j,  1.47522372e-17-3.08517378e-18j,
         5.03492713e-32-8.48165952e-33j, -3.25732403e-17+2.94430289e-17j,
        -4.07661987e-18-3.20159352e-18j,  1.95791824e-33-1.54919509e-34j,
        -1.56855462e-35+9.92831267e-36j, -1.33343058e-33-3.81405787e-33j,
         7.18224619e-46-2.12268394e-46j, -1.63379858e-30-1.20565121e-31j,
         6.39193410e-32+2.34963047e-32j,  4.76858651e-46-3.84142298e-46j],
       [ 0.00000000e+00+0.00000000e+00j, -1.22905816e-16-2.48773839e-17j,
        -4.92487561e-18-6.16455342e-17j, -1.90781307e-16-1.04272957e-16j,
         8.46069512e-17-1.68739148e-16j, -6.20346093e-18-1.67285021e-17j,
        -4.93812614e-16+9.10190901e-17j,  1.42268970e-16-2.88431253e-17j,
        -4.87416064e-17-1.05563863e-16j, -8.50409406e-18+2.48944447e-17j,
        -8.49307725e-17+1.29693183e-1